In [1]:
!pip install flair

In [2]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/NLP/2_Competição/Dataset'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file= 'train.conll',
                              test_file= 'test.conll')

2021-05-20 18:53:17,345 Reading data from /content/drive/MyDrive/NLP/2_Competição/Dataset
2021-05-20 18:53:17,346 Train: /content/drive/MyDrive/NLP/2_Competição/Dataset/train.conll
2021-05-20 18:53:17,347 Dev: None
2021-05-20 18:53:17,348 Test: /content/drive/MyDrive/NLP/2_Competição/Dataset/test.conll


In [10]:
from flair.data import Corpus
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from flair.embeddings import TransformerWordEmbeddings, DocumentPoolEmbeddings
from typing import List
from torch.optim.adam import Adam

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # GloVe embeddings
    WordEmbeddings('pt'),

    # # contextual string embeddings, forward
    PooledFlairEmbeddings('pt-forward', pooling='min'),

    # # contextual string embeddings, backward
    PooledFlairEmbeddings('pt-backward', pooling='min'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        rnn_layers = 2
                                        )

# initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/nerFinal',
              train_with_dev=False,
              learning_rate=0.1,
              min_learning_rate=0.0001,
              mini_batch_size=32,
              max_epochs=80)


2021-05-20 18:43:15,032 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmp2vrw_ub_



 23%|██▎       | 161388544/710528528 [00:08<00:24, 22047294.90B/s]

KeyboardInterrupt: ignored

In [15]:
from transformers import AutoModel, AutoTokenizer, AutoConfig
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
config = AutoConfig.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased', config=config)

tokenizer.save_pretrained('/content/drive/MyDrive/NLP/2_Competição/resultados')
model.save_pretrained('/content/drive/MyDrive/NLP/2_Competição/resultados')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
from flair.data import Corpus
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from flair.embeddings import TransformerWordEmbeddings, DocumentPoolEmbeddings
from typing import List
from torch.optim.adam import Adam
from flair.trainers import ModelTrainer
import torch
import torchvision
from flair.models import SequenceTagger

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

use_context = 16
# hf_model = 'xlm-roberta-large'

hf_model = 'neuralmind/bert-base-portuguese-cased'

# embeddings = TransformerWordEmbeddings('/content/drive/MyDrive/NLP/2_Competição/resultados')


embeddings = TransformerWordEmbeddings(
    model=hf_model,
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
)

tagger: SequenceTagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)

trainer = ModelTrainer(tagger, corpus, optimizer=torch.optim.AdamW)
from torch.optim.lr_scheduler import OneCycleLR

context_string = '+context' if use_context else ''

trainer.train(f"resources/flert",
              learning_rate=5.0e-6,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=5,
              scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.,
              )

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2021-05-20 19:53:58,868 ----------------------------------------------------------------------------------------------------
2021-05-20 19:53:58,871 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29794, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): D

RuntimeError: ignored

In [ ]:
final = pd.read_csv("/content/resources/taggers/nerFinal/test.tsv", sep = ' ', skip_blank_lines=False,header=None)

In [ ]:
final2 = final

In [ ]:
final2 = final2.drop(columns=[1,2])

In [ ]:
final2['Tag'] = final[2]

In [ ]:
final2 = final2.drop(columns=[0])

In [ ]:
final2.index

RangeIndex(start=0, stop=14079, step=1)

In [ ]:
final2["Id"] = pd.Series(np.arange(start=1, stop=14080, step=1), final2.index)

In [ ]:
cols = final2.columns.tolist()
print(cols)
cols = cols[-1:] + cols[:-1]
print(cols)
final2 = final2[cols]

['Tag', 'Id']
['Id', 'Tag']


In [ ]:
final2 = final2.dropna()

In [ ]:
final2.head(30)

,Id,Tag
0,1,O
1,2,O
2,3,O
4,5,O
5,6,O
6,7,O
7,8,O
8,9,O
10,11,O
11,12,O


In [ ]:
final2.to_csv('/content/drive/MyDrive/NLP/2_Competição/resultados/stackedWePtStackedLSTM.csv',index=False)

In [ ]:
from google.colab import files
files.download('/content/resources/taggers/example-ner/final-model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>